# EVALUACIÓN DEL MODELO

## IMPORTACIÓN DE LIBRERÍAS

In [ ]:
import nest_asyncio

nest_asyncio.apply()

from llama_index.core.evaluation import generate_question_context_pairs

from llama_index.core.evaluation import generate_question_context_pairs
from llama_index.core.evaluation import RetrieverEvaluator
from llama_index.llms.gemini import Gemini


import os
import pandas as pd

from src.utils import load_api_key
from src.preprocessing_text import check_and_load_index, load_nodes


/Users/mmolinaalvarez/Library/Caches/pypoetry/virtualenvs/rag-tfm-BaNwZdzT-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [71]:
API_KEY_PATH = ("google_api_key.txt")
DATA_PATH = ("data/text")

api_key = load_api_key("/Users/mmolinaalvarez/Desktop/tfm_valley_2025_g3/google_api_key.txt")
if not api_key:
    raise ValueError("Error: No se pudo cargar la API Key.")
os.environ["GOOGLE_API_KEY"] = api_key

llm = Gemini(model="models/gemini-2.0-flash-exp", api_key=api_key)
nodes = load_nodes()
vector_index = check_and_load_index()
query_engine = vector_index.as_query_engine(llm=llm)
qa_file_path = "/Users/mmolinaalvarez/Desktop/tfm_valley_2025_g3/evaluation/q&a.pkl"
retriever = vector_index.as_retriever(similarity_top_k=3)

Índice persistido encontrado, cargando el índice.


## CREACIÓN Q&A

In [72]:
import pickle  
# Verificar si el archivo ya existe
if os.path.exists(qa_file_path):
    print("Archivo de preguntas y respuestas encontrado y cargado")
    with open(qa_file_path, 'rb') as file:
        qa_dataset = pickle.load(file) 
else:
    print("Archivo de preguntas y respuestas no encontrado. Generando datos...")
    qa_dataset = generate_question_context_pairs(
        nodes=nodes,
        llm=llm,
        num_questions_per_chunk=1
    )
    # Guardar el archivo generado en formato pickle
    with open(qa_file_path, 'wb') as f:
            pickle.dump(qa_dataset, f)
    print("Archivo de preguntas y respuestas guardado en formato pickle en:", qa_file_path)
    with open(qa_file_path, 'rb') as file:
        qa_dataset = pickle.load(file) 
    print("Archivo de preguntas y respuestas creado y cargado")

Archivo de preguntas y respuestas encontrado y cargado


## EVALUACIÓN DE MÉTRICAS

In [39]:
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    ["mrr", "hit_rate"], retriever=retriever
)

In [73]:
def split_queries(queries, chunk_size):
    """Divide un diccionario de queries en fragmentos más pequeños."""
    query_items = list(queries.items())  # Convertimos en lista de tuplas (id, query)
    return [dict(query_items[i:i + chunk_size]) for i in range(10, len(query_items), chunk_size)]

# Definir tamaño de fragmento
chunk_size = 149  # Ajusta según la cuota permitida
query_chunks = split_queries(qa_dataset.queries, chunk_size)

# Verificar fragmentación
print(f"Total de fragmentos: {len(query_chunks)}")

Total de fragmentos: 2


In [74]:
import asyncio

async def evaluate_dataset_in_chunks(retriever_evaluator, qa_dataset, chunk_size=5, workers=2):
    """Evalúa el dataset en fragmentos y une los resultados."""

    # Dividir queries en fragmentos
    query_chunks = split_queries(qa_dataset.queries, chunk_size)

    eval_results = []
    
    # Función interna para evaluar un chunk con manejo de errores
    async def evaluate_chunk(chunk):
        chunk_dataset = qa_dataset.model_copy(update={"queries": chunk})  # Crear dataset con queries limitadas
        try:
            return await retriever_evaluator.aevaluate_dataset(chunk_dataset, workers=workers)
        except Exception as e:
            print(f"Error evaluando un chunk: {e}")
            return []  # Devolver lista vacía en caso de error

    # Ejecutar todas las evaluaciones en paralelo (limitado por workers)
    tasks = [evaluate_chunk(chunk) for chunk in query_chunks]
    chunk_results = await asyncio.gather(*tasks)

    # Unir todos los resultados
    for result in chunk_results:
        eval_results.extend(result)

    return eval_results  # <-- Aquí se devuelve eval_results

# Ejecutar la evaluación
async def main():
    eval_results = await evaluate_dataset_in_chunks(retriever_evaluator, qa_dataset, chunk_size=10, workers=1)
    print(f"Evaluación completada con {len(eval_results)} resultados.")
    return eval_results  # <-- Si quieres usar eval_results después

# Llamar a la función principal
final_results = asyncio.run(main())

Error evaluando un chunk: 429 Quota exceeded for quota metric 'Batch Embed Content API requests' and limit 'Batch embed contents request limit per minute for a region' of service 'generativelanguage.googleapis.com' for consumer 'project_number:584898313510'. [reason: "RATE_LIMIT_EXCEEDED"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
metadata {
  key: "quota_metric"
  value: "generativelanguage.googleapis.com/batch_embed_contents_requests"
}
metadata {
  key: "quota_location"
  value: "europe-west4"
}
metadata {
  key: "quota_limit"
  value: "BatchEmbedContentsRequestsPerMinutePerProjectPerRegion"
}
metadata {
  key: "quota_limit_value"
  value: "150"
}
metadata {
  key: "consumer"
  value: "projects/584898313510"
}
, links {
  description: "Request a higher quota limit."
  url: "https://cloud.google.com/docs/quotas/help/request_increase"
}
]
Error evaluando un chunk: 429 Quota exceeded for quota metric 'Batch Embed Content API requ

In [75]:
def display_results(name, final_results):
    """Display results from evaluate."""

    metric_dicts = []
    for final_results in final_results:
        metric_dict = final_results.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    hit_rate = full_df["hit_rate"].mean()
    mrr = full_df["mrr"].mean()

    metric_df = pd.DataFrame(
        {"Retriever Name": [name], "Hit Rate": [hit_rate], "MRR": [mrr]}
    )

    return metric_df

In [77]:

display_results("Gemini Embedding Retriever", final_results)

,Retriever Name,Hit Rate,MRR
0,Gemini Embedding Retriever,0.0,0.0


## FAITHFULNESS Y RELEVANCE INDIVIDUAL

In [11]:

pro1_5 = Gemini(model="models/gemini-1.5-pro", api_key=api_key, temperature=0)

flash1_5 = Gemini(model="models/gemini-1.5-flash", api_key=api_key, temperature=0)

flash2 = Gemini(model="models/gemini-2.0-flash-exp", api_key=api_key, temperature=0)


In [12]:
from llama_index.core.evaluation import FaithfulnessEvaluator
faithfulness_pro1_5 = FaithfulnessEvaluator(llm=pro1_5)

In [13]:
queries = list(qa_dataset.queries.values())
eval_query = queries[1]
response_vector = query_engine.query(eval_query)

In [14]:
eval_result = faithfulness_pro1_5.evaluate_response(response=response_vector)

In [15]:
eval_result.score

1.0

In [16]:
# Evaluar la fidelidad de las respuestas generadas por cada modelo
eval_result_pro1_5 = FaithfulnessEvaluator(llm=pro1_5).evaluate_response(response=response_vector)
eval_result_flash1_5 = FaithfulnessEvaluator(llm=flash1_5).evaluate_response(response=response_vector)
eval_result_flash2 = FaithfulnessEvaluator(llm=flash2).evaluate_response(response=response_vector)

# Crear una tabla con los resultados
results = {
    "Model": ["pro1_5", "flash1_5", "flash2"],
    "Faithfulness Passing": [eval_result_pro1_5.passing, eval_result_flash1_5.passing, eval_result_flash2.passing]
}

results_df = pd.DataFrame(results)
results_df

,Model,Faithfulness Passing
0,pro1_5,True
1,flash1_5,True
2,flash2,True


In [17]:
from llama_index.core.evaluation import RelevancyEvaluator

relev_result_pro1_5 = RelevancyEvaluator(llm=pro1_5).evaluate_response(query=eval_query, response=query_engine.query(eval_query))
relev_result_flash1_5 = RelevancyEvaluator(llm=flash1_5).evaluate_response(query=eval_query, response=query_engine.query(eval_query))
relev_result_flash2 = RelevancyEvaluator(llm=flash2).evaluate_response(query=eval_query, response=query_engine.query(eval_query))


# # Crear una tabla con los resultados
results = {
    "Model": ["pro1_5", "flash1_5", "flash2"],
    "Relevancy Passing": [relev_result_pro1_5.passing, relev_result_flash1_5.passing, relev_result_flash2.passing]
}

results_df = pd.DataFrame(results)
results_df

,Model,Relevancy Passing
0,pro1_5,True
1,flash1_5,True
2,flash2,True


In [18]:
from llama_index.core.evaluation import BatchEvalRunner

# Let's pick top 10 queries to do evaluation
batch_eval_queries = queries[:10]

# Initiate BatchEvalRunner to compute FaithFulness and Relevancy Evaluation.
runner = BatchEvalRunner(
    {"faithfulness": faithfulness_pro1_5, "relevancy": RelevancyEvaluator(llm=pro1_5)},
    workers=8,
)

# Compute evaluation
eval_results = await runner.aevaluate_queries(
    query_engine, queries=batch_eval_queries
)

ResourceExhausted: 429 Quota exceeded for quota metric 'Batch Embed Content API requests' and limit 'Batch embed contents request limit per minute for a region' of service 'generativelanguage.googleapis.com' for consumer 'project_number:584898313510'. [reason: "RATE_LIMIT_EXCEEDED"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
metadata {
  key: "quota_metric"
  value: "generativelanguage.googleapis.com/batch_embed_contents_requests"
}
metadata {
  key: "quota_location"
  value: "europe-west4"
}
metadata {
  key: "quota_limit"
  value: "BatchEmbedContentsRequestsPerMinutePerProjectPerRegion"
}
metadata {
  key: "quota_limit_value"
  value: "150"
}
metadata {
  key: "consumer"
  value: "projects/584898313510"
}
, links {
  description: "Request a higher quota limit."
  url: "https://cloud.google.com/docs/quotas/help/request_increase"
}
]

In [17]:
faithfulness_score = sum(result.passing for result in eval_results['faithfulness']) / len(eval_results['faithfulness'])

faithfulness_score

0.9

In [18]:
relevancy_score = sum(result.passing for result in eval_results['relevancy']) / len(eval_results['relevancy'])

relevancy_score


1.0

In [19]:
import random

# Configuración de los modelos
pro1_5 = Gemini(model="models/gemini-1.5-pro", api_key=api_key, temperature=0)
flash1_5 = Gemini(model="models/gemini-1.5-flash", api_key=api_key, temperature=0)
flash2 = Gemini(model="models/gemini-2.0-flash-exp", api_key=api_key, temperature=0)

# Evaluador de fidelidad
faithfulness_evaluator = FaithfulnessEvaluator()

# Lista de modelos a evaluar
models = {
    "pro1_5": pro1_5,
    "flash1_5": flash1_5,
    "flash2": flash2
}

# Asegurarse de que la carpeta 'evaluation' existe
os.makedirs("evaluation", exist_ok=True)

# Lista para almacenar los resultados
results = []
selected_queries = random.sample(list(qa_dataset.queries.values()), min(1, len(qa_dataset.queries)))


# Evaluar cada modelo
for model_name, model in models.items():
    # Generar respuestas para cada consulta en el conjunto de datos
    for query in selected_queries:
        response_vector = query_engine.query(query)
        eval_result = faithfulness_evaluator.evaluate_response(response=response_vector, llm=model)
        results.append({
            "Model": model_name,
            "Query": query,
            "Faithfulness Passing": eval_result.passing
        })

# Crear un DataFrame con los resultados
results_df = pd.DataFrame(results)

# Guardar el DataFrame en un archivo CSV en la carpeta 'evaluation'
faithfulness_path = os.path.join("evaluation", "faithfulness_results.csv")
results_df.to_csv(faithfulness_path, index=False, encoding='utf-8')

# print(f"Resultados de la evaluación de fidelidad guardados en: {output_path}")
print(results_df)

KeyboardInterrupt: 

In [54]:
from llama_index.core.evaluation import RelevancyEvaluator
import random

# Evaluador de relevancia
relevancy_evaluator = RelevancyEvaluator()

# Lista para almacenar los resultados
results_relevancy = []
selected_queries = random.sample(list(qa_dataset.queries.values()), min(5, len(qa_dataset.queries)))

# Evaluar cada modelo
for model_name, model in models.items():
    for query in selected_queries:
        response_vector = query_engine.query(query)  # ✅ Ahora usa `query` en vez de `eval_query`
        
        # ✅ Pasamos `query` explícitamente en evaluate_response
        eval_result = relevancy_evaluator.evaluate_response(query=query, response=response_vector, llm=model)
        
        results_relevancy.append({
            "Model": model_name,
            "Query": query,
            "Relevancy Passing": eval_result.passing
        })

# Crear un DataFrame con los resultados
results_relevancy_df = pd.DataFrame(results_relevancy)

# Guardar el DataFrame en un archivo CSV en la carpeta 'evaluation'
relevancy_path = os.path.join("evaluation", "relevancy_results.csv")
results_relevancy_df.to_csv(relevancy_path, index=False, encoding='utf-8')

# print(f"Resultados de la evaluación de fidelidad guardados en: {output_path}")
print(results_relevancy)

## BATCH PARA FAITHFULNESS Y RELEVANCE

In [67]:
from llama_index.core.evaluation import BatchEvalRunner, RelevancyEvaluator, FaithfulnessEvaluator
import random
import pandas as pd
import os

# Definir el nombre del archivo CSV donde se guardarán los resultados
CSV_FILENAME = "batch_evaluation_results.csv"
SCORES_CSV_FILENAME = "model_scores.csv"

# Definir modelos a evaluar
pro1_5 = Gemini(model="models/gemini-1.5-pro", api_key=api_key, temperature=0)
flash1_5 = Gemini(model="models/gemini-1.5-flash", api_key=api_key, temperature=0)
flash2 = Gemini(model="models/gemini-2.0-flash-exp", api_key=api_key, temperature=0)

# Lista de modeloså
models = {
    "pro1_5": pro1_5,
    "flash1_5": flash1_5,
    "flash2": flash2
}

# Seleccionar 10 consultas aleatorias
selected_queries = random.sample(list(qa_dataset.queries.values()), min(3, len(qa_dataset.queries)))

# Evaluadores por modelo
evaluators_per_model = {}
for model_name, model in models.items():
    evaluators_per_model[model_name] = {
        "faithfulness": FaithfulnessEvaluator(llm=model),  # Evaluador de fidelidad específico del modelo
        "relevancy": RelevancyEvaluator(llm=model)  # Evaluador de relevancia específico del modelo
    }

# Lista para almacenar resultados de todos los modelos
all_results = []
model_scores = []  # Para almacenar los scores por modelo




In [68]:


for model_name, evaluators in evaluators_per_model.items():
    print(f" Evaluando modelo: {model_name}")

    # Configurar BatchEvalRunner con 8 workers para evaluación paralela
    runner = BatchEvalRunner(evaluators, workers=8)

    # Ejecutar la evaluación de manera asíncrona
    eval_results = await runner.aevaluate_queries(query_engine, queries=selected_queries)

    # Calcular los scores de fidelidad y relevancia
    faithfulness_score = sum(result.passing for result in eval_results["faithfulness"]) / len(eval_results["faithfulness"])
    relevancy_score = sum(result.passing for result in eval_results["relevancy"]) / len(eval_results["relevancy"])

    model_scores.append({
        "Model": model_name,
        "Faithfulness Score": faithfulness_score,
        "Relevancy Score": relevancy_score
    })

    # Convertir los resultados en un formato adecuado
    for query, eval_data in eval_results.items():
        row = {"Model": model_name, "Query": query}

        #  Si eval_data es una lista, convertirla a un diccionario usando el nombre de la métrica como clave
        if isinstance(eval_data, list):
            eval_data = {evaluator_name: result for evaluator_name, result in zip(evaluators.keys(), eval_data)}

        #  Ahora recorremos cada métrica de evaluación (faithfulness, relevancy)
        for eval_name, eval_result in eval_data.items():
            row[eval_name] = eval_result.passing  
        
        all_results.append(row)

 Evaluando modelo: pro1_5
 Evaluando modelo: flash1_5
 Evaluando modelo: flash2


In [70]:
# Convertir los resultados a DataFrame
df_results = pd.DataFrame(all_results)

# Guardar los resultados de cada consulta en CSV
df_results.to_csv(CSV_FILENAME, index=False, encoding="utf-8")
print(f"✅ Resultados guardados en {CSV_FILENAME}")

# Convertir los scores de cada modelo en DataFrame
df_scores = pd.DataFrame(model_scores)

# # Guardar los scores en otro CSV
SCORES_CSV_FILENAME = "model_scores.csv"
df_scores.to_csv(SCORES_CSV_FILENAME, index=False, encoding="utf-8")
print(f"✅ Scores guardados en {SCORES_CSV_FILENAME}")

# Mostrar los resultados y los scores
print("🔹 Resultados por consulta:")
print(df_results)

print("\n🔹 Scores por modelo:")
print(df_scores)

✅ Resultados guardados en batch_evaluation_results.csv
✅ Scores guardados en model_scores.csv
🔹 Resultados por consulta:
      Model         Query  faithfulness  relevancy
0    pro1_5  faithfulness          True       True
1    pro1_5     relevancy          True       True
2  flash1_5  faithfulness          True       True
3  flash1_5     relevancy          True       True
4    flash2  faithfulness          True       True
5    flash2     relevancy          True       True

🔹 Scores por modelo:
      Model  Faithfulness Score  Relevancy Score
0    pro1_5                 1.0              1.0
1  flash1_5                 1.0              1.0
2    flash2                 1.0              1.0
